<a href="https://colab.research.google.com/github/guidobarra/pyhton-GPU/blob/main/BarraQuelca_GuidoAlberto_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 Introducción

El siguiente ejemplo se multiplicara dos matrices A y B, el resultado de la multiplicacion se guardara en la matriz C.
La multiplicacion de la matrices tiene muchas aplicaciones, una de ellas es en la generacion del movimientos de un objeto en los videojuegos como por ejemplo, la rotacion, el giro o la traslacion se los puede representar mediante una matriz. Al multiplicar algunas de las matrices mensionadas con la matriz de posicion del objeto, se obtiene el movimiento del mismo. 

El desarrollo teorico de la multiplicacion de matrices se mostrara a continuacion.

C = A.B
\begin{pmatrix}a_{11}&\cdots &a_{1n}\\\vdots &\ddots &\vdots \\a_{m1}&\cdots &a_{mn}\end{pmatrix}\begin{pmatrix}b_{11}&\cdots &b_{1p}\\\vdots &\ddots &\vdots \\b_{n1}&\cdots &b_{np}\end{pmatrix}\begin{pmatrix}a_{11}b_{11}+\cdots +a_{1n}b_{n1}&\cdots &a_{11}b_{1p}+\cdots +a_{1n}b_{np}\\\vdots &\ddots &\vdots \\a_{m1}b_{11}+\cdots +a_{mn}b_{n1}&\cdots &a_{m1}b_{1p}+\cdots +a_{mn}b_{np}\end{pmatrix}

La multiplicacion de matrices se puede realizar si la dimensiones de las matrices coinciden, por ejemplo una matriz A de dimension mxn y una matriz B de dimension nxp, se puede realizar la operacion de multiplicacion porque sus dimensiones coinciden, la cantidad de columnas de la matriz A es n y la cantidad de filas de la matriz B es n, la matriz del resultado C tiene la dimension de mxp. De forma simplificada la multiplicacion de matrices se realizas mediante la multiplicacion de los elementos de las filas de la matriz A con los elementos de las columnas de la matriz B, se suma cada multiplicacion y el resultado es un elemento de la matriz C,  debido a esto se valida las dimensiones de las matrices para su multiplicacion.


Para realizar la multipliacion se utilizara la siguiente ecuacion, la cual sale del desarrollo teorico.
<center>$C[f][c]=A[f][0]xB[0][c] + ....... +A[f][k]xB[k][c]$</center>
cuando k sea:
<center>$ k=cantColumnasMatrizA=cantFilasMatrizB$</center>

EL objetivo es enseñar el funcionamiento del Lenguaje Python, CUDA y el manejo de la operacion de matriz a bajo nivel. El ejemplo es ilustrativo para entededer los multi hilos de la GPU en dos dimenciones, basicamente cada hilo realiza la misma operacion de multiplicar las filas por las columnas y el resultado se guarda en la matriz C.

---
# 2 Armado del ambiente
Toma las dimesiones de las matrices A Y B, la deja disponible al contexto de ejecuciòn del cuaderno colab.

In [17]:
#@title # 2.1 Parámetros de ejecución
#@markdown ### Especifique las dimensiones de la matrices:

try: 
  fila_mat_a =  10#@param {type:"integer"}
  colum_mat_a =  10#@param {type:"integer"}

  fila_mat_b =  10#@param {type:"integer"}
  colum_mat_b =  10#@param {type:"integer"}

except Exception as e:
  print("Error de ingresar los parametros")
  print("Error debido a: ", e.__class__)
  print(e)








---
## 2.2 validar los parametros

In [18]:
#A.B
try:
  if fila_mat_a < 0 or colum_mat_a < 0:
    print("ERROR LAS DIMENCIONES DE LA MATRIZ A TIENE QUE SER MAYOR A 0")
  elif fila_mat_b < 0 or colum_mat_b < 0:
    print("ERROR LAS DIMENCIONES DE LA MATRIZ B TIENE QUE SER MAYOR A 0")
  elif colum_mat_a != fila_mat_b:
    print("ERROR LAS DIMENCIONES DE LAS MATRIZ NO PERMITEN REALIZAR LA MULTIPLICACION")
  else: 
    print("OK LAS MATRICES SE PUEDEN MULTIPLICAR")
except Exception as e:
  print("Oops Ocurrio una error!")
  print("Error debido a: ", e.__class__)
  print(e)

OK LAS MATRICES SE PUEDEN MULTIPLICAR


---
## 2.3 Instala en el cuaderno el módulo CUDA de Python.

In [9]:
!pip install pycuda

---
# 3 Desarrollo
Ejecución del algoritmo de multiplicacion de matrices secuencial y paralelo.

---
## 3.1 Desarrollo Secuencial
Ejecución del algoritmo de multiplicacion de matrices en forma secuencial utilizando la CPU.

In [20]:
try:
  %matplotlib inline
  from datetime import datetime
  tiempo_total = datetime.now()
  import numpy
  
  
  # --------------------------------------------
  # Definición de función que transforma el tiempo en  milisegundos 
  tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0
  # --------------------------------------------
  
  tiempo_definicion_matrices = datetime.now()
  
  # CPU - Defino la memoria de la matriz A en cpu.
  matriz_a = numpy.random.randn(fila_mat_a, colum_mat_a)*10
  matriz_a = matriz_a.astype(numpy.int32())
  
  # CPU - Defino la memoria de la matriz B en cpu.
  matriz_b = numpy.random.randn(fila_mat_b, colum_mat_b)*10
  matriz_b = matriz_b.astype(numpy.int32())
  
  # CPU - Defino la memoria de la matriz resultado C en cpu.
  matriz_c = numpy.random.randn(fila_mat_a, colum_mat_b)*0
  matriz_c = matriz_c.astype(numpy.int32())
  matriz_c_secuencial = numpy.random.randn(fila_mat_a, colum_mat_b)*0
  matriz_c_secuencial = matriz_c_secuencial.astype(numpy.int32())
  
  tiempo_definicion_matrices = datetime.now() - tiempo_definicion_matrices
  
  tiempo_multiplicacion_secuencial = datetime.now()
  
  for f in range(fila_mat_a):
    for c in range(colum_mat_b):
      for inter in range(colum_mat_a):
        matriz_c_secuencial[f][c] += matriz_a[f][inter]*matriz_b[inter][c]
  
  tiempo_multiplicacion_secuencial = datetime.now() - tiempo_multiplicacion_secuencial
  tiempo_total = datetime.now() - tiempo_total

  print("\n**MULTIPLICACION DE MATRICES UTILIZANDO CPU**\n")
  print("Tiempo de definicion de matrices: ", tiempo_en_ms( tiempo_definicion_matrices ), "[ms]" )
  print("Tiempo CPU  : ", tiempo_en_ms( tiempo_multiplicacion_secuencial ), "[ms]" )
  print("Tiempo TOTAL: ", tiempo_en_ms( tiempo_total ), "[ms]" )
  print("Matiz A:\n", matriz_a)
  print("Matiz B:\n", matriz_b)
  print("Matiz C:\n", matriz_c_secuencial)
except Exception as e:
  print("Oops Ocurrio una error!")
  print("Error debido a: ", e.__class__)
  print(e)


**MULTIPLICACION DE MATRICES UTILIZANDO CPU**

Tiempo de definicion de matrices:  0.127 [ms]
Tiempo CPU  :  1.239 [ms]
Tiempo TOTAL:  1.377 [ms]
Matiz A:
 [[-24   0   7  18 -20   5   0  -1  -4 -11]
 [-17 -20  17 -11 -19  -6  -3   7  -1   2]
 [ 11  14 -19 -12   9  20  -2   3 -22   1]
 [  0   8   4  10  -6  19   4  -2  -2  -1]
 [ -7  13   0  -3   0 -26   7   0  -2 -12]
 [-15   2  -5 -12   0   6  -8   0   9   0]
 [  5  -5  13  16   2  -5  18 -17  -9   2]
 [ -1   1  -6   8   0   8  -4   6   1  25]
 [-13   9  -2   4   8  -3  18 -15   6  -5]
 [  6  -4   9 -13 -10  -6   6  -4  11   7]]
Matiz B:
 [[  0   8 -18   9 -17  -8  14  11   4  -9]
 [  2 -17   4  -1   6 -18   4   3  -9  17]
 [  4 -10 -16  10   6   2  -1  -7   3 -11]
 [ -7  -7  -7 -11   7  -2  -3  -5   4 -10]
 [  1  23   2   3   6   8 -10  -7 -15   2]
 [ -3  -6 -12   9   9  -3 -19   8   1 -15]
 [ -2   8  10  -9 -16   4   0  -5  21 -13]
 [  6   0   9 -20   6  15   4 -28  -8   3]
 [  2  -1  12   3  -6  11  12  -1  14   3]
 [ 12  -5   0   

---
## 3.2 Desarrollo Paralelo
Ejecución del algoritmo de multiplicacion de matrices en forma paralela utilizando la GPU.

In [21]:
try:
  %matplotlib inline
  from datetime import datetime
  tiempo_total = datetime.now()

  import numpy
  import pycuda.driver as cuda
  import pycuda.autoinit
  from   pycuda.compiler import SourceModule

  # --------------------------------------------
  # Definición de función que transforma el tiempo en  milisegundos
  tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0
  # --------------------------------------------

  tiempo_definicion_matrices = datetime.now()

  # CPU - Defino la memoria de la matriz A en cpu.
  matriz_a = numpy.random.randn(fila_mat_a, colum_mat_a)*10
  matriz_a = matriz_a.astype(numpy.int32())

  # CPU - Defino la memoria de la matriz B en cpu.
  matriz_b = numpy.random.randn(fila_mat_b, colum_mat_b)*10
  matriz_b = matriz_b.astype(numpy.int32())

  # CPU - Defino la memoria de la matriz resultado C en cpu.
  matriz_c = numpy.random.randn(fila_mat_a, colum_mat_b)*0
  matriz_c = matriz_c.astype(numpy.int32())
  matriz_c_secuencial = numpy.random.randn(fila_mat_a, colum_mat_b)*0
  matriz_c_secuencial = matriz_c_secuencial.astype(numpy.int32())

  tiempo_definicion_matrices = datetime.now() - tiempo_definicion_matrices

  tiempo_reserva_memoria_GPU = datetime.now()
  # CPU - reservo la memoria GPU.
  matriz_a_Gpu = cuda.mem_alloc(matriz_a.nbytes)
  matriz_b_Gpu = cuda.mem_alloc(matriz_b.nbytes)
  matriz_c_Gpu = cuda.mem_alloc(matriz_c.nbytes)
  tiempo_reserva_memoria_GPU = datetime.now() - tiempo_reserva_memoria_GPU

  tiempo_copia_memoria_GPU = datetime.now()
  # GPU - Copio la memoria al GPU.
  cuda.memcpy_htod(matriz_a_Gpu, matriz_a)
  cuda.memcpy_htod(matriz_b_Gpu, matriz_b)
  cuda.memcpy_htod(matriz_c_Gpu, matriz_c)
  tiempo_copia_memoria_GPU = datetime.now() - tiempo_copia_memoria_GPU

  #CPU - Defino la funcion kernel que ejecutará en GPU
  module = SourceModule("""
  __global__ void multiplicar(int ancho, int alto, int inter, int *matriz_a , int *matriz_b, int *matriz_c)
  {
      // Calculo las coordenadas del Thread en dos dimensiones.
      int idx = threadIdx.x + blockIdx.x*blockDim.x;
      int idy = threadIdx.y + blockIdx.y*blockDim.y;

      // Verifico que los Thread, esten dentro de las dimensiones de la imagen.
      if( idx < ancho && idy < alto ) {
        int indice = idy+(idx*ancho);
        int i = 0;
        int resul = 0;

        //
        while (i<inter) {
          //multiplico fila por columna y sumo el resultado de cada componente
          resul += matriz_a[idx*inter + i]*matriz_b[idy + i*inter];
          i++;
        }
        //paso el resultado obtenido a la componente de la matriz
        matriz_c[indice] = resul;
      }
  }

  """)

  # CPU - Genero la función kernel.
  kernel = module.get_function("multiplicar")

  dim_hilo_x = 16
  dim_bloque_x = numpy.int( (fila_mat_a+dim_hilo_x-1) / dim_hilo_x )

  dim_hilo_y = 19
  dim_bloque_y = numpy.int( (colum_mat_b+dim_hilo_y-1) / dim_hilo_y )

  print( "Thread: [",
        dim_hilo_x,
        ",",
        dim_hilo_y,
        " ], Bloque : [",
        dim_bloque_x,
        ",",
        dim_bloque_y,
        "]" )
  print( "Total de Thread: [",
        dim_hilo_x*dim_bloque_x,
        ",",
        dim_hilo_y*dim_bloque_y,
        " ]",
        " = ",
        dim_hilo_x*dim_bloque_x*dim_hilo_y*dim_bloque_y )

  tiempo_multiplicacion_paralela = datetime.now()

  kernel( numpy.int32(fila_mat_a),
          numpy.int32(colum_mat_b),
          numpy.int32(colum_mat_a),
          matriz_a_Gpu,
          matriz_b_Gpu,
          matriz_c_Gpu,
          block=( dim_hilo_x, dim_hilo_y, 1 ),
          grid=(dim_bloque_x, dim_bloque_y,1) )

  tiempo_multiplicacion_paralela = datetime.now() - tiempo_multiplicacion_paralela

  # GPU - Copio el resultado desde la memoria GPU.
  cuda.memcpy_dtoh( matriz_c, matriz_c_Gpu )
  tiempo_total = datetime.now() - tiempo_total

  print("\n**MULTIPLICACION DE MATRICES UTILIZANDO GPU**\n")
  print("Tiempo de definicion de matrices: ", tiempo_en_ms( tiempo_definicion_matrices ), "[ms]" )
  print("Tiempo de reserva de matrices: ", tiempo_en_ms( tiempo_reserva_memoria_GPU ), "[ms]" )
  print("Tiempo de copia de matrices: ", tiempo_en_ms( tiempo_copia_memoria_GPU ), "[ms]" )
  print("Tiempo GPU  : ", tiempo_en_ms( tiempo_multiplicacion_paralela ), "[ms]" )
  print("Tiempo TOTAL: ", tiempo_en_ms( tiempo_total ), "[ms]" )
  print("Matiz A:\n", matriz_a)
  print("Matiz B:\n", matriz_b)
  print("Matiz C:\n", matriz_c)
except Exception as e:
  print("Oops Ocurrio una error!")
  print("Error debido a: ", e.__class__)
  print(e)

Thread: [ 16 , 19  ], Bloque : [ 1 , 1 ]
Total de Thread: [ 16 , 19  ]  =  304

**MULTIPLICACION DE MATRICES UTILIZANDO GPU**

Tiempo de definicion de matrices:  2.08 [ms]
Tiempo de reserva de matrices:  0.301 [ms]
Tiempo de copia de matrices:  0.064 [ms]
Tiempo GPU  :  0.092 [ms]
Tiempo TOTAL:  5.813 [ms]
Matiz A:
 [[  3   0   4  -4  -8  -1   9   4   6  -1]
 [ -4  -7   0  -2  25   1  27 -16 -12   2]
 [  8   5   0   9  -5 -12 -12   7   6  -2]
 [ 17 -14   7 -16  -4 -14   5   0   4   0]
 [  4 -10   3   7  -8  -5   8  13 -10  10]
 [-12   4   0   6   8  -8   0   0   9  13]
 [  1   8 -15  10 -12   3  -9   0 -12  -2]
 [ -4  -3   8   8   0   7  -6  -5  -4  -6]
 [-10  -7 -11   6   5   0   9   3   0   0]
 [  1  20  13  -3   1   9  -4   1 -14  -1]]
Matiz B:
 [[  0   8  10   1  -3   0 -10 -18   5  14]
 [ 24   3   1   5  -7  -1  -4   1  -1   0]
 [-14 -12  14   0  -1  -3  -2  -5   9 -19]
 [ -6 -13  -2   5   9 -22 -17  -4   0  -8]
 [ -1 -15  14  -9   0   1  -6 -13 -12  15]
 [-12   3   0   2  21  -5 

---
# 4 Tabla de pasos


 Procesador | Funciòn | Detalle
------------|---------|----------
CPU      | pip install pycuda     | Instala en el cuaderno los driver de CUDA para Python.
CPU      |  import                | Importa los módulos.
CPU      |  datetime.now()        | Toma el tiempo actual.
CPU      |numpy.random.randn(n, m)| Crear una matrices de dimensiones nxm con numeros ramdoms.
CPU      |  ar.astype(np.int32()) | Convierte los elemementos del array ar en elementos de tipo int 32 bit.
**GPU**  |  cuda.mem_alloc()      | Reserva la memoria para las matrices en GPU.
**GPU**  |  cuda.memcpy_htod()    | Copio los valores en crudo de las matrices al GPU.
CPU      |  SourceModule()        | Posee el còdigo del kernel.
CPU      |  module.get_function() | convierte el texto del kernel en funcion de Python.
CPU      | dim_hilo_x, dim_hilo_y | Calcula las dimensiones para la ejecuciòn de 2D.
**GPU**  |  kernel()              | Ejecuta el kernel en GPU, enviando los parametros.
CPU      |  print()               | Informa con algun mensaje.
CPU      | cuda.memcpy_dtoh()     | Copia desde la memoria GPU al CPU.

---
# 5 Conclusiones

Se hicieron varias pruebas con matrices de pocos elementos, 9 = 3x3, y con matrices de muchas elementos, 62500 = 250x250. Se observo que para matrices pequeñas el tiempo de ejecucion de la forma secuencial utilizando la CPU y el tiempo de ejecucion de la forma paralela utilizando la GPU, son aproximadamente iguales y en ocaciones el tiempo de ejecucion total de la GPU es mayor que el tiempo de ejecucion total de la CPU, pareceria que no estaria mejorando el tiempo de ejecucion utilizando la GPU. Sin embargo al momento de tener matrices con muchos elementos se observo que el tiempo de ejecucion de la multiplicacion de matrices utilizando la GPU se mantiene constante o a lo sumo aumenta levemente, miestras que el tiempo de ejecucion de la multiplicacion de matrices utilizando la CPU aumenta significativamente cada vez que aumenta las dimenciones de las matrices.
Se concluye que la utilizacion de la GPU baja el tiempo de ejecucion de la multiplicacion de matrices en comparacion con la CPU siempre que se multiplique matrices con muchos elementos aproximadamente 100 = 10x10

---
# 6 Bibliografía

[1] Multiplicacion de matrices, conceptual: [Pagina economipedia](https://economipedia.com/definiciones/multiplicacion-de-matrices.html)

[2] Introducción a Python: [Página Colab](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/Python_Basico.ipynb) 

[3] Mulplicacion de matrices. Aplicaciones y Desarrollo: [PDF](http://docs.uprb.edu/deptmate/material%20suplementario/CIME/7mo%20a%209no/Matrices%20y%20sus%20Aplicaciones.pdf)

[4] Canal Derivando, Aplicaciones de Matrices: [Video](https://www.youtube.com/watch?v=9FKFgNQktkU)

